In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import time
from pandas import ExcelWriter
import re
import pyodbc
import asyncio
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pdb
#pdb.set_trace()
from ipywidgets import IntProgress
from IPython.display import display
import os
from selenium.webdriver.support import expected_conditions
from IPython.display import clear_output
from selenium.common.exceptions import WebDriverException
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from sc_header import createDriver,config_moneda,extract_json_objects

In [2]:
TIEMPO_DE_ESPERA = 5.5
IMG_AUX = 'https://www.forever21.com/images/1_front_750/00381934-04.jpg'
start_ = datetime.datetime.now()
fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')

In [3]:
browser = createDriver()

browser.get('https://www.forever21.com/us/shop')

url_base = "https://www.forever21.com"



In [4]:
ipath = 'LINKS_F21.xlsx'
LINKS = pd.read_excel(ipath)

In [5]:
LINKS = LINKS[LINKS['TIPO']=='WOMEN + ACC'].reset_index(drop=True)

In [6]:
config_moneda(browser)

In [7]:
LINKS = LINKS[LINKS['CATEGORIA']!='View All']

In [12]:
lista_productos = []
ex_color = 0
for index,link in LINKS.iterrows():
    try:
        browser.get(link['HREF'])
    except WebDriverException:
        
        browser.quit()
        browser = createDriver()
        time.sleep(1)
        continue
        
    config_moneda(browser)
    tipologia = WebDriverWait(browser, 50).until(EC.presence_of_element_located((By.XPATH,'//*[@id="multi_submenu_title"]/h1'))).text
    
    
    max_page = max([int(i.text) for i in browser.find_elements_by_css_selector('#top-pager > ul > li') if i.text.isnumeric()])
               

    for page in range(max_page):

        #cantidad de productos dentro de la página.
        product_link = browser.find_elements_by_css_selector('a.item_slider.product_link')
        
        if browser.current_url == 'https://www.forever21.com/us/shop':
                break
        for indices, product in enumerate(product_link):
            
            try:
                URL = product.get_attribute('href')
            except:
                continue
        
            try:
                browser.execute_script(f"document.getElementsByClassName('qv square')[{indices}].click();")
            except:
                continue
            
            if browser.current_url == 'https://www.forever21.com/us/shop':
                break
            
            
            try:
                ELEMENT_COLORS = WebDriverWait(browser,30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,'#colorButton > li')))
                #ELEMENT_COLORS = browser.find_element_by_id('colorButton').find_elements_by_tag_name('li')
            except:
                time.sleep(3.5)
                continue

            for item, valor in enumerate(ELEMENT_COLORS):
                
                #browser.execute_script(f"document.getElementsByClassName('color')[{index_articulo}].getElementsByTagName('li')[{item}].getElementsByTagName('span')[0].getElementsByTagName('img')[0].click();")
                
                try:
                    if valor.is_displayed():
                        
                        valor.click()
                    #browser.execute_script(f"document.getElementsByClassName('ws_100 swatch row mobile_swatch pt_10')[0].getElementsByClassName('color')[0].getElementsByTagName('li')[{item}].click();")
                except:
                    try:
                        browser.implicitly_wait(10)
                        browser.execute_script(f"document.getElementsByClassName('ws_100 swatch row mobile_swatch pt_10')[0].getElementsByClassName('color')[0].getElementsByTagName('li')[{item}].click();")
                        ex_color += 1 
                    except:
                        break
                        
                
                
                NOMBRE_COLOR = valor.find_element_by_tag_name('img').get_attribute('alt')


                TALLES = browser.find_element_by_xpath('//*[@id="sizeButton"]').find_elements_by_tag_name('li')
                for TALLE in TALLES:

                    #TALLE BLOQUEADO ES OOS
                    if TALLE.find_element_by_tag_name('span').get_attribute('class') != "oos":
                        

                        NOMBRE_TALLE = TALLE.find_element_by_tag_name('span').text

                        # IMG PRODUCTO
                        
                        for _ in range(15):
                            try:
                                IMG = WebDriverWait(browser,50).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#modelImage_1_front_750 > img'))).get_attribute('src')
                            except:
                                time.sleep(2)
                            if IMG != 'https://www.forever21.com/us/shop/_common/images/c_loading.gif':
                                break
                        
                        # DESCRIPCION
                        #DESCRIPCION = browser.find_element_by_class_name('p_name').text
                        
                        DESCRIPCION = browser.execute_script(f"var x = document.getElementsByClassName('p_name')[{indices}].textContent; return x")

                        # PRECIO
                        PRECIO = browser.find_element_by_css_selector('.p_price.pt_5.ws_100.row.mb_10').text
#                        browser.execute_script("var x = document.getElementsByClassName('p_price pt_5 ws_100 row mb_10')[0].textContent; return x").strip()
                        # CODIGO
                        try:
                            CODIGO = browser.find_element_by_css_selector('.t_mid.pt_10').text.split()[-1]
                        except:
                            
                            codigo = browser.execute_script("var x = document.getElementsByClassName('t_mid pt_10')[0].textContent; return x")
                            CODIGO = codigo.split()[-1]
                            

                        lista_productos.append([CODIGO,
                                                NOMBRE_TALLE,
                                                NOMBRE_COLOR,
                                                'WOMEN',
                                                DESCRIPCION,
                                                PRECIO,
                                                IMG,
                                                URL,
                                                tipologia,
                                                page])

            
        #NEXTPAGE
        time.sleep(.8)
        try:
##            browser.execute_script("document.getElementsByClassName('p_next')[0].click();")
            #jquery click next page 
            browser.execute_script("$('#top-pager li span.p_next').click();")
        except:
            break

In [206]:
browser.quit()

In [9]:
browser.current_url

'https://www.forever21.com/us/shop'

In [15]:
df = pd.DataFrame(lista_productos)

In [16]:
len(df)

7189

In [17]:
df.rename(columns={0:'sku',
                   1:'talle',
                   2:'color',
                   3:'sexo',
                   4:'descripcion',
                   5:'precio',
                   6:'url_jpg',
                   7:'url',
                   8:'categoria'},inplace=True)
#df.drop_duplicates()

In [18]:
df.rename(columns={9:'drop'},inplace=True)

In [19]:
df.drop('drop',axis=1,inplace=True)

In [20]:
df.drop_duplicates(inplace=True)

In [21]:
len(df)

7189

In [22]:
df['categoria'].unique()

array(['New Arrivals', "Women's Tops", "Women's Dresses",
       "Women's Bottoms", "Women's Jackets & Coats",
       "Women's Sweaters & Cardigans", "Women's Denim", 'Lingerie',
       "Women's Activewear & Workout Clothes", "Women's Swimsuits",
       'Jumpsuits & Rompers', 'Graphic Tees for Women', 'Bodysuits',
       'Sets'], dtype=object)

In [23]:
df.to_excel(f'./Salida/F21WOMEN{fecha}.xlsx')

In [49]:
#df[df['talle']=='S']

In [42]:
df.loc[df['talle']=='S']

pandas.core.frame.DataFrame

In [34]:
df[df['url_jpg'] == 'https://www.forever21.com/us/shop/_common/images/c_loading.gif']['url'][10]

'https://www.forever21.com/us/shop/catalog/product/f21/shoes/2000401063'

In [45]:
soup = BeautifulSoup(requests.get('https://www.forever21.com/us/shop/catalog/product/f21/top_blouses/2000393792').text,'html.parser')

In [51]:
scripts = soup.find_all('script',type='text/javascript')

In [75]:
XX = scripts[38].text.split('var ')[3].split('brand =')[0]

In [104]:
for flag_color in extract_json_objects(XX):
    pass


In [105]:
for aux_color in flag_color['OnsaleVariants']:
    print(aux_color['ColorName'])
    print(aux_color['ImageFileName'].split('.')[0])

BLACK
00393792-05
WHITE
00393792-05
BLUE
00393792-05
ORANGE
00393792-05


In [114]:
soup


<!DOCTYPE html>

<html class="no-js" lang="en" prefix="og: http://ogp.me/ns#">
<head>
<!-- CCPA S-->
<!-- UsableNet S -->
<script async="" src="https://forever21.usablenet.com/pt/start" type="text/javascript"></script>
<!-- UsableNet E -->
<!-- OneTrust S -->
<script>
            var GTMid = 'GTM-M5K6LBN', GAid = 'UA-233210-1',
                SFEndPoint = 'https://100023399.collect.igodigital.com/collect.js', SalesforceID = '100023399',
                controller = 'Catalog'.toLowerCase(),
                action = 'Product'.toLowerCase(),
                category = 'top_blouses',
                productId = "2000393792",
                UseBloomreach = 'True',
                CountryCode = 'US',
                DynamicYieldID = 8771357,
                Consent = [],
                showBoldchatButton = 'True';
        </script>
<!-- OneTrust Cookies Consent Notice start -->
<script charset="UTF-8" data-domain-script="cacbd7a7-0bbb-4f9c-b832-dc1f83a3560a-test" src="https://cdn.cookiel

In [120]:
browser.quit()
browser = createDriver()
browser.get(LINKS['HREF'][0])

In [121]:
print(browser.current_url)

https://www.forever21.com/us/shop/catalog/category/f21/women-new-arrivals


In [138]:
LINKS.head()

,CATEGORIA,HREF,TIPO
0,New Arrivals,https://www.forever21.com/us/shop/catalog/cate...,WOMEN + ACC
1,Tops,https://www.forever21.com/us/shop/catalog/cate...,WOMEN + ACC
2,Dresses,https://www.forever21.com/us/shop/catalog/cate...,WOMEN + ACC
3,Bottoms,https://www.forever21.com/us/shop/catalog/cate...,WOMEN + ACC
4,Jackets & Outerwear,https://www.forever21.com/us/shop/catalog/cate...,WOMEN + ACC


In [140]:
hrefs_list = []
for index,row in LINKS.iterrows():
    try:
        browser.get(row['HREF'])
    except:
        
        browser.quit()
        time.sleep(1)
        browser = createDriver()
        browser.get(row['HREF'])
    
    
    productos = browser.find_elements_by_css_selector('a.item_slider.product_link')
    
    for indice,producto in enumerate(productos):

        
        hrefs_list.append([indice+1,
                           producto.get_attribute('href'),
                           row['CATEGORIA'],
                           row['HREF'],
                           row['TIPO']
        ])
    #NEXTPAGE
    time.sleep(.8)
    try:
        #jquery click next page 
        browser.execute_script("$('#top-pager li span.p_next').click();")
    except:
        break

1545

In [160]:
aux = pd.DataFrame(hrefs_list)
aux = aux.drop_duplicates(1).reset_index(drop=True)
hrefs_list = aux.values.tolist()

In [ ]:
def scrape_url(pos_aux,url_aux,pagina_aux):
    
    lista_auxiliar = []
    soup = BeautifulSoup(requests.get(url_aux).text,'html.parser')
    
    metadatos = soup.find_all('script',type='application/json')[1]
    metadatos = json.loads(metadatos.text)
    
    for data in metadatos['variants']:
        lista_auxiliar.append([pos_aux,
                               metadatos['id'],#ID
                               metadatos['title'], #DESCRIPCION
                               metadatos['published_at'], #FECHA DE LANZAMIENTO
                               metadatos['vendor'], #MARCA
#                               [i for i in metadatos['tags'] if i != 'oferta'], #DESPUES DE SCRAPEAR, VERIFICAR QUE SOLO TRAIGA UN ELEMENTO
                               data['title'], #talle)
                               data['sku'], #SKU
                               data['available'], #DISPO
                               [i.text.strip() for i in soup.find_all(class_='product-single__prices product-single__prices--policy-enabled')], #PRECIOS
                               soup.find_all('noscript')[-1].img['src'], #TEST
                               #[i.get('src') for i in soup.find_all('img') if i.get('src').find('1024x1024') != -1][0], #IMG
                               url_aux, #URL
                               pagina_aux, #PAGINA_SCRAPER
                              ])
    
    return lista_auxiliar    


#PAGINA_SCRAPER

def scrape_batch(url_chunk):
    chunk_resp = []
    for url in url_chunk:
        chunk_resp.append(scrape_url(url[0],url[1],url[2]))
    return chunk_resp



batch_size = 5

url_chunks = [urllist[x:x+batch_size] for x in range(0, len(urllist), batch_size)]
items = []
for url_chunk in url_chunks:
    items.append(scrape_batch(url_chunk))


# In[180]:


new_list = aplanar_lista(items)


In [165]:
datetime.datetime.today()

datetime.datetime(2020, 1, 31, 18, 3, 1, 419040)

15

23.3

6